In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys

module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from generator import RoadNetwork
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import pandas as pd
import networkx as nx
import numpy as np
from tqdm import tqdm

from models import SRN2VecModel
import json

In [2]:
network = RoadNetwork()
network.load("../../osm_data/porto")

In [39]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SRN2VecModel(None, device, network)

In [59]:
model.generate_data(n_shortest_paths=100, number_negative=3, window_size=900, save_batch_size=100)

100%|██████████| 1/1 [10:14<00:00, 614.46s/it]


In [4]:
paths = model.generate_shortest_paths(
            network.line_graph, n_shortest_paths=2
)

100%|██████████| 11331/11331 [00:04<00:00, 2342.42it/s]


In [42]:
%lprun -f model.extract_pairs model.generate_train_pairs(network, paths, 900, 7)

Timer unit: 1e-06 s

Total time: 82.5101 s
File: /dstore/home/pheinemeyer/Road-Network-Embedding-Generator/models/srn2vec.py
Function: extract_pairs at line 90

Line #      Hits         Time  Per Hit   % Time  Line Contents
    90                                               def extract_pairs(
    91                                                   self,
    92                                                   info_length,
    93                                                   info_highway,
    94                                                   node_list,
    95                                                   node_paths,
    96                                                   window_size,
    97                                                   number_negative,
    98                                               ):
    99     11331      13911.0      1.2      0.0          res = []
   100     22662     138555.0      6.1      0.2          orig_lengths = np.array(
   101     1133

In [29]:
#%load_ext line_profiler
%lprun -f model.extract_pairs model.generate_train_pairs(network, paths, 600, 3)

Timer unit: 1e-06 s

Total time: 2983.19 s
File: /dstore/home/pheinemeyer/Road-Network-Embedding-Generator/models/srn2vec.py
Function: extract_pairs at line 79

Line #      Hits         Time  Per Hit   % Time  Line Contents
    79                                               def extract_pairs(self, info, node_list, node_paths, window_size, number_negative):
    80     11331      14823.0      1.3      0.0          res = []
    81     22662     157941.0      7.0      0.0          orig_lengths = np.array(
    82     11331      46284.0      4.1      0.0              [0] + [len(x) for x in node_paths]
    83                                                   ).cumsum()  # lengths of orginal sequences in flatted
    84     11331      80400.0      7.1      0.0          flatted = list(chain.from_iterable(node_paths))
    85                                                   # flat array and save indices -> loc on all and reshape after
    86                                                   # g

In [10]:
print(model.data[:40])

[[0, 2, 1, 1], [0, 6, 1, 1], [0, 15, 1, 1], [0, 17, 1, 0], [0, 403, 1, 0], [2, 6, 1, 1], [2, 15, 1, 1], [2, 17, 1, 0], [2, 403, 1, 0], [6, 15, 1, 1], [6, 17, 1, 0], [6, 403, 1, 0], [15, 17, 1, 0], [15, 403, 1, 0], [17, 403, 1, 1], [15, 410, 1, 0], [15, 413, 1, 1], [15, 416, 1, 0], [17, 410, 1, 1], [17, 413, 1, 0], [17, 416, 1, 0], [403, 410, 1, 1], [403, 413, 1, 0], [403, 416, 1, 0], [410, 413, 1, 0], [410, 416, 1, 0], [413, 416, 1, 0], [17, 1749, 1, 0], [17, 1752, 1, 0], [17, 6482, 1, 0], [403, 1749, 1, 0], [403, 1752, 1, 0], [403, 6482, 1, 0], [410, 1749, 1, 0], [410, 1752, 1, 0], [410, 6482, 1, 0], [413, 1749, 1, 0], [413, 1752, 1, 0], [413, 6482, 1, 0], [416, 1749, 1, 0]]


In [25]:
n1 = list(network.line_graph.nodes)[413]
n2 = list(network.line_graph.nodes)[1749]
network.gdf_edges.loc[[n1, n2]]

,,,osmid,oneway,highway,length,lanes,ref,maxspeed,bridge,name,width,access,junction,tunnel,area,fid,geometry,highway_enc
u,v,key,,,,,,,,,,,,,,,,,
126580884,126580298,0,13713844,1,motorway_link,62.279,None,None,None,None,None,None,None,None,None,None,561,"LINESTRING (-8.64759 41.16445, -8.64791 41.164...",2
497690739,497690735,0,450854292,1,secondary,59.811,None,None,None,None,Rua de São João Bosco,None,None,None,None,None,4241,"LINESTRING (-8.64854 41.16463, -8.64898 41.164...",6


In [28]:
network.line_graph.get_edge_data(list(network.line_graph.nodes)[2], list(network.line_graph.nodes)[6])

{}

In [28]:
if not os.path.isfile("srn2vec-traindata.json"):
    with open("srn2vec-traindata.json", "w") as fp:
        json.dump(model.data, fp)
else:
    with open("srn2vec-traindata.json", "r") as fp:
        a = json.load(fp)
    with open("srn2vec-traindata.json", "w") as fp:
        a.append(model.data)
        json.dump(a, fp)

In [60]:
with open("srn2vec-traindata.json", "r") as fp:
    a = json.load(fp)

In [61]:
print(len(a))
b = np.unique(np.array(a), axis=0)
print(b[:10, :])

3188608
[[ 0  1  1  1]
 [ 0  2  1  1]
 [ 0  5  1  0]
 [ 0  6  1  1]
 [ 0  7  0  0]
 [ 0  9  1  0]
 [ 0 11  1  1]
 [ 0 12  1  1]
 [ 0 13  0  0]
 [ 0 14  1  0]]
